In [2]:
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# Упражнение 1

In [9]:
from pandas.tseries.holiday import AbstractHolidayCalendar, Holiday, nearest_workday, EasterMonday
from pandas.tseries.offsets import DateOffset, Day

data = pd.read_csv('/home/lycoris/Загрузки/data-20191112T1252-structure-20191112T1247.csv')
data.loc[22]

Год/Месяц                                                                                   2021
Январь                                                    1,2,3,4,5,6,7,8,9,10,16,17,23,24,30,31
Февраль                                                             6,7,13,14,20,21,22*,23,27,28
Март                                                                    6,7,8*,13,14,20,21,27,28
Апрель                                                                 3,4,10,11,17,18,24,25,30*
Май                                                             1,2,3+,8,9,10+,15,16,22,23,29,30
Июнь                                                               5,6,11*,12,13,14+,19,20,26,27
Июль                                                                    3,4,10,11,17,18,24,25,31
Август                                                                   1,7,8,14,15,21,22,28,29
Сентябрь                                                                   4,5,11,12,18,19,25,26
Октябрь                       

In [ ]:
def gettimestamps(d):
    result = []
    extras = []
    year = d[0]
    for month in range(1, 13):
        days = d[month]
        for day in days.split(','):
            if day[-1] in '*+':
                extras.append(day[-1])
                day = day[:-1]
            else:
                extras.append(None)
            result.append(pd.Timestamp(year, month, int(day)))
    return result, extras


timestamps = [gettimestamps(data.loc[i]) for i in range(len(data))]
timestamps = pd.DataFrame({
    'timestamps': [ts for ts, _ in timestamps],
    'extras': [extra for _, extra in timestamps]
}, index=range(1999, 2026))

RULES = []
for i in timestamps.index:
    RULES.extend(
        pd.tseries.holiday.Holiday(
            str(ts),
            month=ts.month,
            year=ts.year,
            day=ts.day,
        )
        for j, ts in enumerate(timestamps.loc[i, 'timestamps'])
        if timestamps.loc[i, 'extras'][j] != '*'
    )

class RussianBusinessCalendar(pd.tseries.holiday.AbstractHolidayCalendar):
    rules = RULES

cal = RussianBusinessCalendar()
offset = pd.tseries.offsets.CustomBusinessDay(calendar=cal)

In [ ]:
class RussianBusinessCalendar(AbstractHolidayCalendar):
    start_date = datetime.datetime(1999, 1, 1)
    end_date = datetime.datetime(2019, 12, 31)
    rules = [Holiday('Рабочая суббота', month=3, day=7, year=2020,
                     offset=Day(1))]
    
russian_busday = pd.offsets.CustomBusinessDay(weekmask='Mon Tue Wed Thu Fri', 
                                              calendar=RussianBusinessCalendar())

In [ ]:
df = pd.DataFrame(
    {},
    index=pd.date_range(pd.Timestamp(1999, 1, 1), pd.Timestamp(2025, 12, 31)),
    columns=[
        'выходной', 'праздник', 'предпраздничный рабочий день',
        'последний день месяца', 'последний рабочий день месяца',
        'пн', 'вт', 'ср', 'чт', 'пт', 'сб', 'вс',
        'январь', 'февраль', 'март', 'апрель', 'май', 'июнь', 'июль',
        'август', 'сентябрь', 'октябрь', 'ноябрь', 'декабрь',
        'Предновогодний выходной день'
    ]
)

df.loc[:, :] = 0

df.loc[cal.holidays(df.index[0], df.index[-1]), 'выходной'] = 1

for year in timestamps.index:
    tss = timestamps.loc[year, 'timestamps']
    extras = timestamps.loc[year, 'extras']
    df.loc[[ts for j, ts in enumerate(tss) if extras[j] == '*'], 'предпраздничный рабочий день'] = 1
    
for day in df.index:
    if day.is_month_end:
        df.loc[day, 'последний день месяца'] = 1
    df.loc[day, df.columns[day.day_of_week + 5]] = 1
    df.loc[day, df.columns[day.month + 11]] = 1
    n = offset.apply(day)
    if df.loc[day, 'выходной']:
        if n.year != day.year:
            df.loc[day, 'Предновогодний выходной день'] = 1
    elif n.month != day.month:
        df.loc[day, 'последний рабочий день месяца'] = 1
        
df.loc[pd.date_range(pd.Timestamp(2021, 1, 1), pd.Timestamp(2021, 12, 31))]

# Упражнение 2


In [13]:
ts = pd.read_csv('/home/lycoris/python_data_analysis_2021/data/sber_stocks.csv', usecols=['CLOSE', 'TRADEDATE'], 
                 index_col='TRADEDATE', parse_dates=True)
ts = ts.CLOSE


TRADEDATE
2013-03-25     98.79
2013-03-26     97.20
2013-03-27     96.75
2013-03-28     98.59
2013-03-29     98.76
               ...  
2020-01-03    255.00
2020-01-06    253.90
2020-01-08    259.15
2020-01-09    257.99
2020-01-10    258.19
Name: CLOSE, Length: 1712, dtype: float64